In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
# ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-170030
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-170030


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

new_compute_cluster = 'project1'

try:
    compute_target = ComputeTarget(workspace=ws, name=new_compute_cluster)
    print('Found existing compute cluster created via SDK: ', new_compute_cluster)
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, new_compute_cluster, compute_config)
    print('Created a new cluster via SDK.')

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing compute cluster created via SDK:  project1
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-12-29T00:59:11.992000+00:00', 'errors': None, 'creationTime': '2021-12-29T00:53:22.562976+00:00', 'modifiedTime': '2021-12-29T00:53:28.145058+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling ({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200) 
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"
os.makedirs(script_folder, exist_ok=True)
shutil.copy("./train.py", script_folder)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=script_folder,
    compute_target=compute_target,
    entry_script="train.py",
    vm_size="Standard_d2_v2",
    vm_priority="lowpriority"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=4,
    policy=policy,
    estimator=est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_75cd091d-4d4f-4729-9f98-8e625b0418ee
Web View: https://ml.azure.com/runs/HD_75cd091d-4d4f-4729-9f98-8e625b0418ee?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-170030/workspaces/quick-starts-ws-170030&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-12-29T01:37:50.552807][API][INFO]Experiment created<END>\n""<START>[2021-12-29T01:37:51.499980][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-12-29T01:37:52.183919][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_75cd091d-4d4f-4729-9f98-8e625b0418ee
Web View: https://ml.azure.com/runs/HD_75cd091d-4d4f-4729-9f98-8e625b0418ee?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-170030/workspaces/quick-starts-ws-170030&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_75cd091d-4d4f-4729-9f98-8e625b0418ee',
 'target': 'project1',
 'status': 'Completed',
 'startTimeUtc': '2021-12-29T01:37:50.359228Z',
 'endTimeUtc': '2021-12-29T01:40:26.888979Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '923553f4-7b7c-43d2-965c-db41a9418484',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '18',
  'score': '0.9122913505311078',
  'best_child_run_id': 'HD_75cd091d-4d4f-4729-9f98-8e625b0418ee_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg170030.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_75cd091d-4d4f-4729-9f98-8e625b0418ee/azureml-logs

KeyError: 'log_files'

In [5]:
import joblib
# Get your best run and save the model from that run.

hyperdrive_run_best = hyperdrive_run.get_best_run_by_primary_metric()

print('best run details: \n', hyperdrive_run_best.get_details())
print('best run metrics: \n', hyperdrive_run_best.get_metrics())

hyperdrive_model = hyperdrive_run_best.register_model(
    model_name='hyperdrive_run_best',
    model_path='/azureml-logs/model.joblib')

best run details: 
 {'runId': 'HD_75cd091d-4d4f-4729-9f98-8e625b0418ee_1', 'target': 'project1', 'status': 'Completed', 'startTimeUtc': '2021-12-29T01:39:06.526306Z', 'endTimeUtc': '2021-12-29T01:39:39.119662Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '923553f4-7b7c-43d2-965c-db41a9418484', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1', '--max_iter', '150'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'project1', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'Experiment udacity

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files([path])

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

data = pd.concat([x, y], axis=1)
train_data, test_data = train_test_split(data, test_size=0.2)
train_data.to_csv("training/training_df.csv")

ds_training = ws.get_default_datastore()
ds_training.upload(src_dir="training", target_path="training_data")

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds_training, ("training_data/training_df.csv"))])

Uploading an estimated of 2 files
Target already exists. Skipping upload for training_data/train.py
Target already exists. Skipping upload for training_data/training_df.csv
Uploaded 0 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_ds,
    label_column_name="y",
    n_cross_validations=5,
    enable_early_stopping=True,
    enable_onnx_compatible_models=True
)

In [9]:
# Submit your automl run

automl_run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6b83a2f2-78c8-4fca-9255-00bcf2e9fa55,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [11]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
# Retrieve and save your best automl model.

print("autoML best run metrics: \n ", automl_run.get_metrics())

automl_best_run_onnx, automl_fitted_model_onnx = automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx')

autoML best run metrics: 
  {'recall_score_micro': 0.916350531107739, 'precision_score_weighted': 0.9119165817512769, 'norm_macro_recall': 0.5257755504583406, 'precision_score_micro': 0.916350531107739, 'matthews_correlation': 0.5603049418893661, 'precision_score_macro': 0.7986393651917035, 'AUC_macro': 0.9479595263489127, 'accuracy': 0.916350531107739, 'average_precision_score_weighted': 0.9553641842920901, 'f1_score_micro': 0.916350531107739, 'recall_score_macro': 0.7628877752291702, 'average_precision_score_macro': 0.8273661554361398, 'balanced_accuracy': 0.7628877752291702, 'AUC_weighted': 0.9479595263489127, 'weighted_accuracy': 0.9549984082289201, 'average_precision_score_micro': 0.9812339118385193, 'log_loss': 0.22437925850392726, 'AUC_micro': 0.9805934478482824, 'f1_score_weighted': 0.9136858494278144, 'recall_score_weighted': 0.916350531107739, 'f1_score_macro': 0.7790343610471788}
